In [78]:
from bs4 import BeautifulSoup

In [79]:
import requests

In [80]:
url = 'http://effatha.pl/slownik.html'

In [81]:
index = requests.get(url)

In [82]:
bs = BeautifulSoup(index.text, 'html5lib')

In [83]:
entries = bs.find(name='div', attrs={'id': 'slownik'}).find_all(['li'])

In [84]:
from tqdm import tqdm_notebook

In [85]:
# entries_clean = []
# for e in tqdm_notebook(entries):
#     if e.find('h4'):
#         link = e.find('a')
#         if link and link.attrs['href'].startswith('slownik'):
#             entries_clean.append(str(link))

In [86]:
# len(entries_clean)

In [87]:
import re

In [88]:
# entries_clean

In [89]:
# t = entries_clean[0]

In [90]:
# re.findall(r'href="(\S+)"', str(entries_clean[0]))

In [91]:
def get_yt_url(e):
    href = re.findall(r'href="(\S+)"', str(e))[0]
    data = requests.get('http://effatha.pl/' + href).text
    if data:
        iframe = re.findall(r'iframe\s*src="(\S+)"', str(data))[0]
#         iframe = BeautifulSoup(data,'html5lib').find('iframe')
        return iframe
    else:
        print('error')

In [92]:
# urls = [get_yt_url(e) for e in tqdm_notebook(entries_clean)]

In [93]:
# %time movies = p.map(get_yt_url, entries_clean[:10])    

In [94]:
import os

In [95]:
movies = os.listdir('app/movies/')

In [96]:
import unicodedata

In [97]:
dictionary = dict([(unicodedata.normalize('NFC', p.split('-')[0].lower()), p) for p in movies])

In [134]:
dictionary['latać'] = dictionary['latać samolotem']
dictionary['odlecieć'] = dictionary['latać samolotem']

dictionary['jechać'] = dictionary['jeździć']
dictionary['odjechać'] = dictionary['jeździć']
dictionary['odjeżdżać'] = dictionary['jeździć']

In [159]:
[k for k in dictionary.keys() if k.startswith('je')]

['jeśli',
 'jezioro',
 'jest',
 'jego',
 'jesień',
 'jeździć na nartach',
 'jedynka',
 'jelenia góra',
 'jezus chrystus',
 'jezus',
 'jeździć',
 'jehowa',
 'jedzenie2',
 'jeść',
 'jeszcze',
 'jedzenie',
 'jechać',
 'jerozolima',
 'jeleń']

In [160]:
import subprocess

In [161]:
import pexpect
def lemmatize(word):
    child = pexpect.spawn('morfeusz_analyzer', encoding='utf-8')
    child.expect('sgjp \(default\)\r\n')
    child.sendline(word)
    child.readline()
    child.kill
    response = child.readline()
    return response.split(',')[3].split(':')[0]

In [162]:
lemmatize('samochody')

'samochód'

In [163]:
import Levenshtein

In [164]:
def find_synonym(word):
    for k,p in dictionary.items():
        if Levenshtein.ratio(k, word) >= 0.7:
            return k,p
    synonyms = get_synonyms(word)
    for syn in synonyms:
        for k,p in dictionary.items():
            if Levenshtein.ratio(k, syn) >= 0.7:
                return k,p
    return None,None

In [165]:
lemmatize('odjeżdża')

'odjeżdżać'

In [166]:
find_synonym('odjeżdża')

('odjeżdżać', 'jeździć-C1yjqX-AXb0.mp4')

In [167]:
lemmatize('opóźniony')

'opóźnić'

In [168]:
from app.ivr.wdnet import is_known, get_synonyms, are_synonyms

In [169]:
sent = 'pociąg relacji Warszawa Gdańsk jest opóźniony o 10 minut'

In [170]:
sent = 'pociąg odjeżdża za 10 minut'

In [171]:
from nltk import pos_tag
import nltk

In [172]:
pos_tag(sent.split())

[('pociąg', 'NN'),
 ('odjeżdża', 'NN'),
 ('za', 'NN'),
 ('10', 'CD'),
 ('minut', 'NN')]

In [173]:
from moviepy.editor import *

In [174]:
from many_stop_words import get_stop_words

In [175]:
stopwords = get_stop_words('pl')

In [176]:
def word2clip(word):
    if word in stopwords:
        return
    if word not in dictionary:
        lemmatized = lemmatize(word) or word
    else:
        lemmatized = word
    clip = dictionary.get(lemmatized, None)
    if clip:
        print('original: ', word, 'lemmatized: ', lemmatized, 'clip', clip, type(clip))
        return VideoFileClip(os.path.join('app/movies', clip))
    else:
        synonym, clip = find_synonym(word)
        print('original: ', word, 'synnonym: ', synonym, 'clip', clip, type(clip))
        if clip:
            return VideoFileClip(os.path.join('app/movies', clip))

In [177]:
def sent2movie(sent):
    clips = []
    for w in sent.split():
        clip = word2clip(w)
        if clip:
            clips.append(clip.resize((460,460)))
    print(clips)
    return concatenate_videoclips(clips)

In [178]:
movie = sent2movie(sent)

original:  pociąg lemmatized:  pociąg clip pociąg-EMvnOAZopLo.mp4 <class 'str'>
original:  odjeżdża lemmatized:  odjeżdżać clip jeździć-C1yjqX-AXb0.mp4 <class 'str'>
original:  10 lemmatized:  10 clip 10-y_tUPPPcJfc.mp4 <class 'str'>
original:  minut lemmatized:  minuta clip minuta-nnyW_39XwsY.mp4 <class 'str'>
[<moviepy.video.io.VideoFileClip.VideoFileClip object at 0x114840438>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x111d42e48>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x111d42ac8>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x111d42cf8>]


In [35]:
movie.duration

19.379999999999999

In [36]:
# movie = movie.speedx(2)

In [37]:
movie.duration

19.379999999999999

In [38]:
movie.write_videofile('./tmp.mp4')

[MoviePy] >>>> Building video ./tmp.mp4
[MoviePy] Writing audio in tmpTEMP_MPY_wvf_snd.mp3


100%|██████████| 428/428 [00:00<00:00, 777.01it/s]

[MoviePy] Done.
[MoviePy] Writing video ./tmp.mp4



100%|██████████| 485/485 [00:08<00:00, 54.90it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./tmp.mp4 

